In [1]:
from instance_reader import read_instance
import random
from Solucion import Solucion
from funciones_auxiliares import seleccionar_segun_probabilidad
from Low_levels import *
from Hiperheuristica import HiperHeuristica

In [8]:
if __name__ == "__main__":
    # comenzamos leyendo una instancia
    instance = read_instance("datasets/b/instance_0001.txt") # objeto instancia
    n = 0
    Ll1= LL_agregacion_ordenes_random( id=n) # o
    n +=1
    Ll2 = LL_agregacion_pasillos_random(id=n) # o
    n +=1
    Ll3 = LL_agregacion_pasillos_top(id=n) # o
    n +=1
    Ll4 = LL_eliminacion_ordenes_random(id=n) # o
    n +=1
    Ll5 = LL_eliminacion_pasillos_random(id=n) #original
    n +=1
    Ll6 = LL_eliminacion_pasillos_malos(id= n)
    n += 1
    #Ll7 = LL_reparacion_factibilidad(id=6) # o 
    #n +=1
    Ll8 = LL_factibilizar_demanda(id=n)
    n +=1
    Ll9 = LL_factibilizar_UB(id=n)
    n +=1
    Ll10 = LL_factibilizar_LB(id = n)
    n +=1
    Ll11 = LL_agregacion_ordenes_chicas(id=n)
    n +=1
    Ll12 = LL_eliminacion_ordenes_chicas(id = n)
    n +=1
    Ll13 = LL_eliminacion_ordenes_grandes(id=n)
    n +=1
    Ll14 = LL_agregacion_pasillo_orden_top(id = n)
    n +=1
    Ll15 = LL_eliminacion_pasillo_malo(id=n)
    n +=1
    Ll16 = LL_agregacion_orden_chica(id =n)

    low_levels = [Ll1,Ll2,Ll3,Ll4,Ll5,Ll6,Ll8,Ll9,Ll10,Ll11,Ll12,Ll13,Ll15,Ll16]

    # elegimos ordenes al azar y pasillos para comenzar
    orders = random.sample(instance.orders, round(0.2* len(instance.orders)))
    runners = random.sample(instance.runners, len(instance.runners))
    # definimos la solucion vacia
    solucion = Solucion(selected_orders= [], selected_runners = [], instance= instance)


    hiperHeuristica = HiperHeuristica( instancia= instance, V = 0.04, low_levels= low_levels, solucion= solucion, tiempo = 580)
    hiperHeuristica.implementar()

    print(hiperHeuristica.mejor_sol)



Conteo = 1, s_t = 0.00 False, s_c = 0.00 False, s* = 0.00 False, secuencia = [9, 10]
Conteo = 2, s_t = 0.00 False, s_c = 0.00 False, s* = 0.00 False, secuencia = [9]
Conteo = 6, s_t = 0.00 False, s_c = 0.00 False, s* = 0.00 False, secuencia = [13, 6, 7, 11]
Conteo = 7, s_t = 0.00 False, s_c = 0.00 False, s* = 0.00 False, secuencia = [0]
Conteo = 8, s_t = 0.00 False, s_c = 0.00 False, s* = 0.00 False, secuencia = [9]
Conteo = 10, s_t = 0.00 False, s_c = 0.00 False, s* = 0.00 False, secuencia = [2, 3]
Conteo = 11, s_t = 0.00 False, s_c = 0.00 False, s* = 0.00 False, secuencia = [3]
Conteo = 13, s_t = 0.10 False, s_c = 0.10 False, s* = 0.10 False, secuencia = [8, 10]
Conteo = 15, s_t = 16.00 False, s_c = 0.10 False, s* = 0.10 False, secuencia = [12, 12]
Conteo = 16, s_t = 0.10 False, s_c = 0.10 False, s* = 0.10 False, secuencia = [10]
Conteo = 17, s_t = 0.10 False, s_c = 0.10 False, s* = 0.10 False, secuencia = [6]


KeyboardInterrupt: 

In [3]:
hiperHeuristica.S

array([[0.58333333, 0.41666667],
       [0.33333333, 0.66666667],
       [0.8       , 0.2       ],
       [0.5       , 0.5       ],
       [0.75      , 0.25      ],
       [0.66666667, 0.33333333],
       [0.35714286, 0.64285714],
       [0.77777778, 0.22222222],
       [0.33333333, 0.66666667],
       [0.66666667, 0.33333333],
       [0.875     , 0.125     ],
       [0.5       , 0.5       ],
       [0.5       , 0.5       ]])